# Pickomino

solve the pickomino game (rules available [here](https://www.ultraboardgames.com/pickomino/game-rules.php)), with a modified rule: when you chose a dice value, you dont have to take all the dices of this roll with this value

For now it doesnt take into account available dominos, their value (dominos 21 and 25 both mark 1 point), it just try to maximise expected score of dices

In [251]:
from functools import lru_cache
from scipy.special import comb
import numpy as np
from itertools import combinations_with_replacement, product
from collections import Counter
import operator

## setup of the game

In [ ]:
# not used yet
dominos = range(21, 36)
ndices = 8
# symbol and their value in term of score
symbols_values = dict(zip(['1','2','3','4','5','worm'], [1,2,3,4,5,5]))

## roll representation

utils to compute probability of a given roll. A roll is an aggregated view counter {dice: number ot times it appear in the roll}

In [294]:
@lru_cache(None)
def compute_proba(dico_values, normalize=True):
    """
    if normalize: proba
    else: number of ways to get this roll
    """
    # number of combination for a given roll aggregate
    value = np.prod([comb(n, k) for k,n in zip(dico_values, np.cumsum(dico_values[::-1])[::-1])])
    if not normalize:
        return value
    # normalize by number of outcome of a roll
    return value / (len(symbols_values)**sum(dico_values))

def v(dico):
    """
    utils to see the values of a dict as a tuple for caching purpose
    """ 
    return tuple(dico.values())

# confront to formula on a particular roll
bob = {'1': 1, '3': 1, '4': 2,'5': 2,'worm': 2}
print(comb(8,1) * comb(7,1) * comb(6,2) * comb(4,2) * comb(2,2))
print('number of way to make this combination', compute_proba(v(bob), False))
print('proba', compute_proba(v(bob), True))

5040.0
number of way to make this combination 5040.0
proba 0.0030006858710562414


In [177]:
# check probas for each possible roll sums to 1
# combinations_with_replacement generates all possible roll (aggregated)
# our function computes the probability of this roll
sum([ compute_proba(v(Counter(c))) for c in combinations_with_replacement(symbols_values.keys(), 8)])

1.0000000000000027

## more utilities

In [ ]:
def roll_dices(ndices):
    """
    return all rolls and their probabilities for a given number of dices
    """
    return [(Counter(c), compute_proba(tuple(Counter(c).values()))) for c in combinations_with_replacement(symbols_values.keys(), ndices)]

def decisions(used_dices, roll):
    """
    return all possible actions on a given roll
    given allready used symbols
    """
    return [(d, n) for d in roll.keys() if d not in used_dices for n in range(1, 1+roll[d])]

def t(tu):
    """
    utility to have a cachable (hashable) set-like
    """
    return tuple(sorted(tu))


In [300]:
# more probable outcome
# (one of)
tmp = roll_dices(8)
max(tmp, key=operator.itemgetter(1))

(Counter({'1': 2, '2': 2, '3': 1, '4': 1, '5': 1, 'worm': 1}),
 0.006001371742112483)

## game solver

In [247]:
# we only keep track of the score / used dices forreduced signature / better caching
@lru_cache(None)
def compute_mean_score(used_dices, curr_score, n_dices_left, my_last_domino=0):
    # if stop: curr score if we have a worm
    stop_score = curr_score if 'worm' in used_dices else -my_last_domino
    # if no more dices: we have to used the stop score
    if n_dices_left == 0:
        return stop_score
    # average score on all roll outcomes if we keep rolling
    keep_score = 0
    scenarios = roll_dices(n_dices_left) # list of (roll, proba)
    for scenario, proba in scenarios:
        scores_scenario = []
        for dice_value, number_selected in decisions(used_dices, scenario):
            scores_scenario.append(compute_mean_score(
                t(set(used_dices).union([dice_value])),
                curr_score + number_selected * symbols_values[dice_value],
                n_dices_left - number_selected,
                my_last_domino
            ))
        # if no actions possibles: we lose our last domino
        if len(scores_scenario) == 0:
            scores_scenario = [-my_last_domino]
        # the score of the scenario is the maximum on all actions in the given scenario
        score_scenario = max(scores_scenario)
        # the average score of keep playing is incremented by the proba
        #  of the scenario * max score on action of the scenario
        keep_score += proba * score_scenario

    # the final score is the max between stop and keep
    return max(stop_score, keep_score)

In [324]:
def play():

    # initialize tracking variables
    res = {}
    used_dices = tuple([])
    curr_score = 0
    n_dices_left = 8
    my_last_domino = 25
    r = []

    while True:
        # chose a roll with probabilities
        all_rolls = roll_dices(n_dices_left)
        roll = np.random.choice([r[0] for r in all_rolls], p=[r[1] for r in all_rolls])
        print('roll', roll)

        # track all decisions scores
        res = {}
        # impact of each decision on the roll
        for dice_value, number_selected in decisions(used_dices, roll):
            res[(dice_value, number_selected)] = compute_mean_score(
                t(set(used_dices).union([dice_value])),
                curr_score + number_selected * symbols_values[dice_value],
                n_dices_left - number_selected,
                my_last_domino
            )

        # if no decision possible: we lose
        if len(res) == 0:
            print('#############   LOST   #################')
            return 

        # chose decision with better expected outcome
        max_action, max_keep_score = max(res.items(), key=operator.itemgetter(1))

        # apply action
        used_dices = t(set(used_dices).union([max_action[0]]))
        curr_score = curr_score + max_action[1] * symbols_values[max_action[0]]
        n_dices_left = n_dices_left - max_action[1]

        print('best decision', 'take dice \'{}\' {} times'.format(*max_action), ' estimate {}'.format(max_keep_score), ' current {}'.format(curr_score))

        r.append((max_action, curr_score))

        # stop if estimation worse than current
        if (max_keep_score <= curr_score) and ('worm' in used_dices):
            break

    if 'worm' not in used_dices:
        print('#############   LOST   #################')
        return
    return r



In [325]:
play()

roll Counter({'2': 3, 'worm': 2, '1': 1, '3': 1, '4': 1})
best decision take dice 'worm' 2 times  estimate 22.2277299939311  current 10
roll Counter({'3': 2, '2': 1, '4': 1, '5': 1, 'worm': 1})
best decision take dice '3' 2 times  estimate 20.99382716049382  current 16
roll Counter({'1': 2, '2': 1, 'worm': 1})
best decision take dice '1' 2 times  estimate 18  current 18


[(('worm', 2), 10), (('3', 2), 16), (('1', 2), 18)]